In [1]:
# Imports
import subprocess
import os

In [14]:
dir = "1a4g_na"
target = "1A4G"

In [15]:
# Create pdbqt files of receptor and ligand using obabel
!obabel $dir/$target-receptor.pdb -xr -O $dir/$target-receptor.pdbqt
!obabel $dir/$target-ZMR.pdb -O $dir/$target-ZMR.pdbqt



1 molecule converted
1 molecule converted


In [16]:
# in Pymol we get the bounding box of the chosen ligand like this: get_extent ligand_ZMR
# This gives us the min values and max values for x, y, z, from these compute the box centers for docking
min = {'x': -7.638, 'y': 53.013, 'z': -12.316}
max = {'x': 1.912, 'y': 59.198, 'z': -4.813}

# calc box center and size
center_x = (min['x'] + max['x']) / 2
center_y = (min['y'] + max['y']) / 2
center_z = (min['z'] + max['z']) / 2

# calc box size
size_x = (max['x'] - min['x']) + 4
size_y = (max['y'] - min['y']) + 4
size_z = (max['z'] - min['z']) + 4

print("Center X: ", center_x, "Center Y: ", center_y, "Center Z: ", center_z)
print("Size X: ", size_x, "Size Y: ", size_y, "Size Z: ", size_z)

Center X:  -2.863 Center Y:  56.1055 Center Z:  -8.5645
Size X:  13.55 Size Y:  10.185000000000002 Size Z:  11.503


In [18]:
# The ligand and receptor preparation for 1A4G was done following this guide: https://www.cheminformania.com/ligand-docking-with-smina/
# Receptor: 1A4G-receptor.pdbqt
# Ligand: 1A4G-ZMR.pdbqt
# Used parameters for box size and box center found here: https://autodock-vina.readthedocs.io/en/latest/docking_basic.html
cmd = [
    'vina',  
    '--receptor', '1a4g_na/1A4G-receptor.pdbqt',
    '--ligand', '1a4g_na/1A4G-ZMR.pdbqt',
    '--center_x', str(center_x),  # specify box center coordinates (float)
    '--center_y', str(center_y),
    '--center_z', str(center_z),
    '--size_x', str(size_x),
    '--size_y', str(size_y),
    '--size_z', str(size_z),
    '--exhaustiveness', '16',
    '--out', '1a4g_na/vina_docked.pdbqt'
]

subprocess.run(cmd)
print("Docking done via command line")

AutoDock Vina v1.2.7
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [19]:
# Function to extract the energetically most favorable candidate from the vina_docked file
def extract_top_pose(vina_pdbqt_path, output_path):
    with open(vina_pdbqt_path) as f_in, open(output_path, 'w') as f_out:
        write = False
        for line in f_in:
            if line.startswith('REMARK VINA RESULT:'):
                if write:  # we've already found first pose, stop here
                    break
                else:
                    write = True  # start writing pose from here
            if write:
                f_out.write(line)


In [20]:
# Extract top candidate
extract_top_pose('1a4g_na/vina_docked.pdbqt', '1a4g_na/top_pose.pdbqt')
!obabel $dir/top_pose.pdbqt -O $dir/top_pose.pdb


1 molecule converted


In [21]:
# Compute RMSD of the energetically most favourable candidate
!python3 compute_rmsd.py -i $dir/$target-ZMR.pdb -i2 $dir/top_pose.pdb

RMSD: 0.10 Å


In [22]:
# To check if there is a candidate with a better RMSD score, we split the mulit-pose file generated by AutoDock Vine
def split_pdbqt(filename, molecule_pdb):
    with open(filename, 'r') as f:
        lines = f.readlines()

    pose_num = 0
    pose_lines = []
    for line in lines:
        if line.startswith("MODEL"):
            pose_lines = [line]
        elif line.startswith("ENDMDL"):
            pose_lines.append(line)
            pose_num += 1
            out_name = f"{molecule_pdb}_Candidates/pose_{pose_num}.pdbqt"
            with open(out_name, "w") as out_file:
                out_file.writelines(pose_lines)
            print(f"Saved {out_name}")
        else:
            pose_lines.append(line)


split_pdbqt("1a4g_na/vina_docked.pdbqt", "1A4G")


Saved 1A4G_Candidates/pose_1.pdbqt
Saved 1A4G_Candidates/pose_2.pdbqt
Saved 1A4G_Candidates/pose_3.pdbqt
Saved 1A4G_Candidates/pose_4.pdbqt
Saved 1A4G_Candidates/pose_5.pdbqt
Saved 1A4G_Candidates/pose_6.pdbqt
Saved 1A4G_Candidates/pose_7.pdbqt
Saved 1A4G_Candidates/pose_8.pdbqt
Saved 1A4G_Candidates/pose_9.pdbqt


In [23]:
!python3 compute_multiple_rmsd.py -i $dir/$target-ZMR.pdb -i2 1A4G_Candidates/ 

Computing RMSD for 9 poses:
Pose 1: RMSD = 0.10 Å
Pose 2: RMSD = 0.20 Å
Pose 3: RMSD = 0.35 Å
Pose 4: RMSD = 0.96 Å
Pose 5: RMSD = 0.85 Å
Pose 6: RMSD = 0.29 Å
Pose 7: RMSD = 1.04 Å
Pose 8: RMSD = 0.25 Å
Pose 9: RMSD = 1.17 Å

Best RMSD: 0.10 Å at pose 1
